In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

whole_df = pd.read_csv('../data/clean_df.csv', index_col=[0])

In [12]:
whole_df['macro_bhv'].unique()

array(['moving', 'feeding', 'resting', 'grooming', 'social',
       'social-playing', 'agonistic', nan, 'interacting with public',
       'exploring', 'playing', 'sex'], dtype=object)

In [14]:
whole_df.rename(columns={'macro_bhv':'bhv_category'}, inplace=True)

In [15]:
whole_df.loc[whole_df['bhv_category'].isin(['moving', 'resting', 'feeding', 'grooming', 'exploring']), 'macro_bhv'] = 'Individual'
whole_df.loc[whole_df['bhv_category']=='playing', 'macro_bhv'] = 'Ludic'
whole_df.loc[whole_df['bhv_category']=='social-playing', 'macro_bhv'] = 'Social-Ludic'
whole_df.loc[whole_df['bhv_category']=='social', 'macro_bhv'] = 'Social'
whole_df.loc[whole_df['bhv_category']=='agonistic', 'macro_bhv'] = 'Agonistic'
whole_df.loc[whole_df['bhv_category']=='interacting with public', 'macro_bhv'] = 'Public'
whole_df.loc[whole_df['bhv_category']=='sex', 'macro_bhv'] = 'Sexual'




In [16]:
whole_df.head()

,date,subject,period,reg,behavior,receptor,duration,bhv_category,macro_bhv
0,2019-04-15,Karl,pregame,1,cam,[],15.0,moving,Individual
1,2019-04-15,Karl,pregame,1,forr,[],34.0,feeding,Individual
2,2019-04-15,Karl,pregame,1,sen,[],52.0,resting,Individual
3,2019-04-15,Karl,pregame,1,cam,[],5.0,moving,Individual
4,2019-04-15,Karl,pregame,1,sen,[],34.0,resting,Individual


# Histogram for all behaviors

In [11]:
import math
beh=(whole_df['behavior'].unique())
def behavior_histogram(behavior):
    return go.Histogram(x=whole_df[(whole_df['behavior']==behavior)&(whole_df['duration']<150)]['duration'],
    name=behavior,
    marker={'color':'#C5D845'})
fig = make_subplots(rows=4, cols=8, subplot_titles=beh, vertical_spacing=0.2)
for b in range(len(beh)):
    row = math.floor(b/8+1)
    col = b%8+1
    fig.append_trace(behavior_histogram(beh[b]), row, col)
fig.update_layout({'height':600,
                   'showlegend':False, 
                   'plot_bgcolor': 'rgba(21, 47, 18, 255)',
                   'paper_bgcolor': 'rgba(21, 47, 18, 255)',
                   'font':{'color': 'rgba(230,230,230,255)'},
                   'yaxis':{'showgrid':False}
                   })
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False, showgrid=False)
fig.show()

In [109]:
import math
beh=(whole_df['macro_bhv'].unique())
def behavior_histogram(behavior):
    return go.Histogram(x=whole_df[(whole_df['macro_bhv']==behavior)&(whole_df['duration']<150)]['duration'],
    name=behavior,
    marker={'color':'#C5D845'})
fig = make_subplots(rows=4, cols=2, subplot_titles=beh, vertical_spacing=0.2)
for b in range(len(beh)):
    row = math.floor(b/2+1)
    col = b%2+1
    fig.append_trace(behavior_histogram(beh[b]), row, col)
fig.update_layout({'height':600,
                   'showlegend':False, 
                   'plot_bgcolor': 'rgba(21, 47, 18, 255)',
                   'paper_bgcolor': 'rgba(21, 47, 18, 255)',
                   'font':{'color': 'rgba(230,230,230,255)'},
                   'yaxis':{'showgrid':False}
                   })
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False, showgrid=False)
fig.show()

# Boxplot by Behavior and monkey

In [9]:
DATA = whole_df
def box_by_behavior(subjects, behavior):
    if not isinstance(subjects, list):
        subjects=[subjects]
    df = DATA[(DATA['subject'].isin(subjects))&(DATA['macro_bhv']==behavior)]
    fig = px.box(df, x='period', y='duration', color='period', width=500)
    return fig

fig = box_by_behavior(['Storma', 'Hadia', 'Popo', 'Jawie','Karl', 'Lockie'], 'Social')
fig.show()


In [54]:
whole_df.to_csv('../data/clean_df.csv')

# Relative durations

In [64]:
whole_df.head()

,date,subject,period,reg,behavior,receptor,duration,bhv_category,macro_bhv
0,2019-04-15,Karl,pregame,1,cam,[],15.0,moving,Individual
1,2019-04-15,Karl,pregame,1,forr,[],34.0,feeding,Individual
2,2019-04-15,Karl,pregame,1,sen,[],52.0,resting,Individual
3,2019-04-15,Karl,pregame,1,cam,[],5.0,moving,Individual
4,2019-04-15,Karl,pregame,1,sen,[],34.0,resting,Individual


In [3]:
whole_df['rel_duration'] = whole_df.groupby(['date', 'subject', 'reg', 'period'])['duration'].apply(lambda x:x/x.sum())


In [2]:
from queries import *


In [6]:
df = where_subject_and_date(whole_df, 'Karl', '2019-04-15')
df[df['reg']==1]['rel_duration'].sum()

0.9999999999999999

In [7]:
whole_df.to_csv('../data/clean_df.csv')

# Frequencies

Filtrar por mono

In [54]:
freq_popo_game = freq[(freq['subject']=='Popo')]
freq_popo_game.reset_index().head()

,index,date,reg,subject,period,macro_bhv,freq
0,9,2019-04-15,1,Popo,pregame,Individual,1
1,10,2019-04-15,1,Popo,pregame,Social,2
2,11,2019-04-15,1,Popo,pregame,Social-Ludic,2
3,23,2019-04-15,2,Popo,pregame,Individual,8
4,24,2019-04-15,2,Popo,pregame,Ludic,2


Sumar las frecuencias en cada periodo

In [68]:
aaa= freq_popo_game.groupby(['period','macro_bhv'])['freq'].sum().reset_index()
aaa

,period,macro_bhv,freq
0,game,Agonistic,5
1,game,Individual,232
2,game,Ludic,129
3,game,Social,116
4,game,Social-Ludic,34
5,postgame,Individual,225
6,postgame,Ludic,74
7,postgame,Public,3
8,postgame,Social,127
9,postgame,Social-Ludic,35


Crear primero una nueva columna con todo a 0 y después calcular las frecuencias relativas

In [104]:
import numpy as np
aaa['relative_freq'] = 0
def d(x, period):
    sum = x.groupby('period')['freq'].sum()
    x['relative_freq'] += (x['period']==period).astype(int)*x['freq'] / sum[period]

d(aaa,'game')
d(aaa,'pregame')
d(aaa,'postgame')
aaa

,period,macro_bhv,freq,relative_freq
0,game,Agonistic,5,0.009690
1,game,Individual,232,0.449612
2,game,Ludic,129,0.250000
3,game,Social,116,0.224806
4,game,Social-Ludic,34,0.065891
5,postgame,Individual,225,0.484914
6,postgame,Ludic,74,0.159483
7,postgame,Public,3,0.006466
8,postgame,Social,127,0.273707
9,postgame,Social-Ludic,35,0.075431


Diagrama de barras con frecuencias relativas para cada mono

In [106]:
px.bar(aaa, x='period', y='relative_freq', color='macro_bhv', width=400)